In [ ]:
# import libraries and data from reuters, along with testing/training sets. Limiting the most frequently 10k occuring words to help with computing speed

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

In [ ]:
# checking length of training data

len(train_data)

In [ ]:
# checking length of testing data

len(test_data)

In [ ]:
# checking to see that each datapoint is a list of integers for word indices

train_data[10]

In [ ]:
# decoding each datapoint back to english text

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

In [ ]:
# checking to see what the labels are (should be an integer between 0:45)

train_labels[10]

In [ ]:
# turning our lists of integers (train_data/test_data) into lists of tensors using multi-hot encoding

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) # creating a matrix of all zeros
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i,j]=1 
    return results

In [ ]:
# Vectorizing the testing and training data

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
# One-hot encoding the labels 

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [ ]:
# applying the encoding to the labels for both testing & training

y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

In [ ]:
# defining the model with 2 dense layers of 64 and a final layer of 46 since there are 46 potential labels

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

In [ ]:
# compiling the model

model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# setting aside 1000 datapoints for a validation set

x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

In [ ]:
# Training my model with 20 epochs

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [ ]:
# Visualizing the training and validation loss

loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Visualizing the training and validation accuracy

plt.clf()
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# retraining the model to avoid overfitting

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(x_train,
          y_train,
          epochs=9,
          batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results

In [ ]:
# running a baseline test to see what a random classifier would get for accuracy compared to the results above

import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

In [ ]:
# generating predictions for test data

predictions = model.predict(x_test)

In [ ]:
# should be a vector with a length of 46,0 (since there are 46 options for classifications)

predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])

In [ ]:
# if we wanted to encode the labels differently we could cast as an integer tensor:

y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
# since the labels would not be categorical encoding, but rather integer labels a different loss function is needed

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# deliberately implementing an information "bottleneck" due to having an intermediate layer with less than 46 units

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))